In [1]:
from gurobipy import Model, GRB
import numpy as np
import torch
import matplotlib.pyplot as plt

# Toy problem in gurobi

In [ ]:
def solve_gromov_wasserstein(mu, nu, Cx, Cy):
    """
    Solves the Gromov-Wasserstein quadratic program using Gurobi.

    Parameters:
        mu (numpy.ndarray): Source distribution, shape (n,).
        nu (numpy.ndarray): Target distribution, shape (m,).
        G (numpy.ndarray): Quadratic cost matrix, shape (nm, nm).

    Returns:
        numpy.ndarray: Optimal transport matrix T, shape (n, m).
    """
    # Dimensions
    n = len(mu)
    m = len(nu)

    # Create a Gurobi model
    model = Model("Gromov-Wasserstein")

    # Add variables for T (n x m), flattened as a single vector
    T = model.addVars(n, m, lb=0, vtype=GRB.CONTINUOUS, name="T")

    # Flatten T for indexing consistency
    # T_vec = T.flatten()

    # Add marginal constraints
    # Row marginal: sum_j T[i, j] = mu[i]
    for i in range(n):
        model.addConstr(sum(T[i, j] for j in range(m)) == mu[i])

    # Column marginal: sum_i T[i, j] = nu[j]
    for j in range(m):
        model.addConstr(sum(T[i, j] for i in range(n)) == nu[j])

    obj = 0
    for i in range(n):
        for j in range(m):
            for k in range(n):
                for el in range(m):
                    # obj += -2*(Cx[i,j] * Cy[k,el]) * T[i, k] * T[j,el]
                    obj += (Cx[i, j] - Cy[k, el]) ** 2 * T[i, k] * T[j, el]
    # obj = 0.5 * sum(
    #     G[i, j] * T_vec[i] * T_vec[j] for i in range(n * m) for j in range(n * m)
    # )
    model.setObjective(obj, GRB.MINIMIZE)

    # Optimize the model
    model.optimize()

    # Extract solution
    if model.Status == GRB.OPTIMAL:
        T_opt = np.array([[T[i, j].X for j in range(m)] for i in range(n)])
        return T_opt, model
    else:
        print("Optimization failed!")
        return None, None


# Example usage
n = m = 5
mu = np.random.rand(n)
mu /= mu.sum()  # Normalize to sum to 1
nu = np.random.rand(m)
nu /= nu.sum()  # Normalize to sum to 1
points_x = np.random.randn(n, 2)
points_y = np.random.randn(m, 2)


def cdist(x, y):
    return np.sum((x[:, None] - y[None, :]) ** 2, axis=-1)


Cx = cdist(points_x, points_x)
Cy = cdist(points_y, points_y)

T_optimal, model = solve_gromov_wasserstein(mu, nu, Cx, Cy)
print("Optimal Transport Matrix:\n", T_optimal)

Verify objective

# Problem formulations: GW as QP

In [15]:
class Args:
    n_i = 1
    n_j = 4
    n_downsample_pix = 20
    top_k = 3
    exponent = 1
    scheduler = "none"
    element_wise = False
    cost_scale_factor = 1.0


args = Args()
torch_dtype = torch.float32

n_downsample_pix = args.n_downsample_pix
top_k = args.top_k
exponent = args.exponent
scheduler = args.scheduler
element_wise = args.element_wise
cost_scale_factor = args.cost_scale_factor
normalize = True

In [17]:
fname = "/mnt/home/smbp/ceph/smbpchallenge/round2/set2/processed_submissions/submission_23.pt"
submission = torch.load(fname, weights_only=False)
volumes = submission["volumes"].to(torch_dtype)

In [18]:
volumes_i = volumes[: args.n_i]
volumes_j = volumes[: args.n_j]

In [19]:
from gw_weighted_voxels import prepare_volume_and_distance

marginals_i = np.empty((len(volumes_i), top_k))
marginals_j = np.empty((len(volumes_j), top_k))
pairwise_distances_i = np.empty((len(volumes_i), top_k, top_k))
pairwise_distances_j = np.empty((len(volumes_j), top_k, top_k))

for i in range(len(volumes_i)):
    volume_i, pairwise_distance_i = prepare_volume_and_distance(
        volumes_i[i], top_k, n_downsample_pix, exponent, cost_scale_factor, normalize
    )
    marginals_i[i] = volume_i
    pairwise_distances_i[i] = pairwise_distance_i
for j in range(len(volumes_j)):
    volume_j, pairwise_distance_j = prepare_volume_and_distance(
        volumes_j[j], top_k, n_downsample_pix, exponent, cost_scale_factor, normalize
    )
    marginals_j[j] = volume_j
    pairwise_distances_j[j] = pairwise_distance_j

In [ ]:
# def generate_G(Cx, Cy):
#     """
#     Generate the quadratic cost matrix G for the Gromov-Wasserstein problem.

#     Parameters:
#         Cx (numpy.ndarray): Distance matrix for the source space, shape (n, n).
#         Cy (numpy.ndarray): Distance matrix for the target space, shape (m, m).

#     Returns:
#         numpy.ndarray: Quadratic cost matrix G, shape (nm, nm).
#     """
#     n, m = Cx.shape[0], Cy.shape[0]

#     # Initialize the G matrix with zeros
#     G = np.zeros((n * m, n * m))

#     # Fill in G using the formula
#     for i in range(n):
#         for j in range(m):
#             for k in range(n):
#                 for el in range(m):
#                     idx1 = i * m + j  # Flattened index for (i, j)
#                     idx2 = k * m + el  # Flattened index for (k, l)
#                     G[idx1, idx2] = -2*Cx[i, j]*Cy[k,el]
#                     # G[idx1, idx2] = (Cx[i, j] - Cy[k,el])**2

#     return G


gw_distance = np.zeros((len(volumes_i), len(volumes_j)))
transport_plans = np.zeros((len(volumes_i), len(volumes_j), top_k, top_k))
for idx_i in range(len(volumes_i)):
    for idx_j in range(len(volumes_j)):
        print(f"between {idx_i} and {idx_j}...")
        if idx_i <= idx_j:
            print(f"Computing GW distance between {idx_i} and {idx_j}...")

            n = m = top_k
            Cx = pairwise_distances_i[idx_i]
            Cy = pairwise_distances_j[idx_j]

            # Ensure the distance matrices are symmetric and non-negative
            Cx = 0.5 * (Cx + Cx.T)
            Cy = 0.5 * (Cy + Cy.T)
            Cx[Cx < 0] = 0
            Cy[Cy < 0] = 0

            # G = generate_G(Cx, Cy)
            mu = marginals_i[idx_i]
            nu = marginals_j[idx_j]
            T_optimal, model = solve_gromov_wasserstein(mu, nu, Cx, Cy)
            # gw_distance[idx_i, idx_j] = gw_distance[idx_j, idx_i] = model.ObjVal
            gw_distance[idx_i, idx_j] = model.ObjVal
            # gw_distance[idx_i, idx_j] += (np.outer(mu,mu) * Cx**2).sum() + (np.outer(nu,nu) * Cy**2).sum()
            transport_plans[idx_i, idx_j] = T_optimal

In [ ]:
transport_plans

In [ ]:
transport_plans

In [ ]:
transport_plans

In [ ]:
plt.imshow(gw_distance)
plt.colorbar()
gw_distance

In [ ]:
plt.imshow(gw_distance)
plt.colorbar()
gw_distance

# verify Gurobi solver works

# nlopt

In [ ]:
import nlopt
import numpy as np


# Define the objective function
def objective_function(x, grad):
    """
    f(x) = (x - 3)^2
    The gradient is optional; NLopt computes it automatically if not provided.
    """

    return x.T @ np.eye(2) @ x


# Set up the optimizer
opt = nlopt.opt(nlopt.LD_MMA, 2)  # Use COBYLA algorithm with 1 variable
opt.set_min_objective(objective_function)  # Set the objective function
opt.set_lower_bounds([-10, -10])  # Set lower bound for x
opt.set_upper_bounds([10, 10])  # Set upper bound for x
opt.set_xtol_rel(1e-6)  # Set tolerance for convergence

# Initial guess for x
x0 = np.array([0.1, 0.2])

# Run the optimization
try:
    x_optimal = opt.optimize(x0)
    min_value = opt.last_optimum_value()  # Minimum value of the objective function
    print("Optimal x:", x_optimal)
    print("Optimal f(x):", min_value)
except RuntimeError as e:
    print("NLopt failed:", e)

In [ ]:
import nlopt
import numpy as np


# Define the objective function
def objective_function(x, grad):
    """
    f(x) = x^T I x
    The gradient is optional; NLopt computes it automatically if not provided.
    """
    # if grad.size > 0:
    #     grad[:] = 2 * x  # Gradient of x^T I x is 2x
    return x.T @ np.eye(len(x)) @ x


# Set up the optimizer
opt = nlopt.opt(nlopt.LD_MMA, 2)  # Use MMA algorithm with 2 variables
opt.set_min_objective(objective_function)  # Set the objective function
opt.set_lower_bounds([-10, -10])  # Set lower bounds for x
opt.set_upper_bounds([10, 10])  # Set upper bounds for x
opt.set_xtol_rel(1e-6)  # Set tolerance for convergence

# Initial guess for x
x0 = np.array([1, 1])

# Run the optimization
try:
    x_optimal = opt.optimize(x0)
    min_value = opt.last_optimum_value()  # Minimum value of the objective function
    print("Optimal x:", x_optimal)
    print("Optimal f(x):", min_value)
except RuntimeError as e:
    print("NLopt failed:", e)

In [ ]:
import numpy as np
import nlopt


# Define the objective function
def objective_function(x, grad):
    """
    The objective function for the non-convex QP problem.
    """
    # Here, x is a flattened version of the optimal transport matrix
    # Let's assume the quadratic form for simplicity
    G = np.array([[1, 2], [3, 4]])  # Example quadratic cost matrix (should be larger)
    c = np.zeros_like(x)  # Linear term is zero
    return 0.5 * np.dot(x.T, np.dot(G, x)) + np.dot(c.T, x)  # 0.5 x^T G x + c^T x


# Define constraints (row and column marginals)
def constraint_function(x, grad, mu, nu, n, m):
    """
    Constraints for the row and column marginals.
    """
    constraint = np.zeros(n + m)
    # Row constraints: sum_j T[i, j] = mu[i]
    for i in range(n):
        constraint[i] = np.sum(x[i * m : (i + 1) * m]) - mu[i]

    # Column constraints: sum_i T[i, j] = nu[j]
    for j in range(m):
        constraint[n + j] = np.sum(x[j::m]) - nu[j]

    return constraint


# Set up the problem
def solve_non_convex_qp(mu, nu, n, m):
    """
    Solves the Gromov-Wasserstein problem using NLopt for non-convex QP.
    """
    # Initialize the optimization variable (flattened transport matrix T)
    x0 = np.zeros(n * m)  # Initial guess: zero matrix
    lower_bounds = np.zeros_like(x0)  # T[i, j] >= 0

    # Define the optimizer and objective
    opt = nlopt.opt(
        nlopt.LD_MMA, n * m
    )  # LD_MMA is a popular algorithm for non-convex problems
    opt.set_min_objective(objective_function)
    opt.set_lower_bounds(lower_bounds)

    # Set constraints: row and column marginals
    opt.add_inequality_constraint(
        lambda x, grad: constraint_function(x, grad, mu, nu, n, m), 1e-8
    )

    # Solve the problem
    try:
        optimal_transport = opt.optimize(x0)
        return optimal_transport.reshape((n, m))  # Reshape to the transport matrix
    except RuntimeError as e:
        print(f"NLopt error: {e}")
        return None


# Example usage
n, m = 4, 4
mu = np.random.rand(n)
mu /= mu.sum()  # Normalize to sum to 1
nu = np.random.rand(m)
nu /= nu.sum()  # Normalize to sum to 1

# Solve the non-convex QP
T_optimal = solve_non_convex_qp(mu, nu, n, m)
print("Optimal Transport Matrix (T):\n", T_optimal)

In [ ]:
# import numpy as np
# import nlopt


# # Define the Gromov-Wasserstein objective function
# def gromov_wasserstein_objective(T_flat, G_x, G_y, n, m):
#     """
#     Objective function for the Gromov-Wasserstein problem.
#     T_flat is the flattened transport matrix.
#     """
#     p = 2
#     T = T_flat.reshape((n, m))
#     obj_value = 0
#     # Compute the GW cost
#     for i in range(n):
#         for j in range(m):
#             for k in range(n):
#                 for el in range(m):
#                     obj_value += T[i, k] * T[j, el] * (G_x[i, j] - G_y[k, el]) ** p

#     return obj_value


# # Define the constraint function for row and column marginals
# def constraint_function(T_flat, mu, nu, n, m):
#     """
#     Constraints for the row and column marginals.
#     Ensures that row sums are mu and column sums are nu.
#     """
#     T = T_flat.reshape((n, m))
#     row_constraints = np.zeros(n)
#     col_constraints = np.zeros(m)

#     # Row constraints: sum_j T[i, j] = mu[i]
#     for i in range(n):
#         row_constraints[i] = np.sum(T[i, :]) - mu[i]

#     # Column constraints: sum_i T[i, j] = nu[j]
#     for j in range(m):
#         col_constraints[j] = np.sum(T[:, j]) - nu[j]

#     # Combine row and column constraints
#     return np.concatenate([row_constraints, col_constraints])


# # Set up and solve the Gromov-Wasserstein problem
# def solve_gw_problem(G_x, G_y, mu, nu):
#     n, m = G_x.shape  # Sizes of the two sets
#     x0 = np.ones(n * m) / (n * m)  # Initial guess: uniform distribution
#     lower_bounds = np.zeros_like(x0)  # Ensure T[i, j] >= 0

#     # Set up the optimizer
#     opt = nlopt.opt(nlopt.LD_MMA, n * m)  # MMA is a good choice for non-convex problems
#     opt.set_min_objective(
#         lambda T_flat: gromov_wasserstein_objective(T_flat, G_x, G_y, n, m)
#     )
#     opt.set_lower_bounds(lower_bounds)

#     # Add row and column marginal constraints
#     opt.add_inequality_constraint(
#         lambda T_flat: constraint_function(T_flat, mu, nu, n, m), 1e-8
#     )

#     # Solve the problem
#     try:
#         optimal_T_flat = opt.optimize(x0)
#         optimal_T = optimal_T_flat.reshape((n, m))  # Reshape to the transport matrix
#         return optimal_T
#     except RuntimeError as e:
#         print(f"NLopt error: {e}")
#         return None


# # Example usage
# n, m = 4, 4  # Size of the two sets (you can change this)
# # Random distance matrices (G_x and G_y)
# G_x = np.random.rand(n, n)
# G_y = np.random.rand(m, m)

# # Normalize the distance matrices (optional)
# G_x = (G_x + G_x.T) / 2  # Ensure symmetric distance matrix
# G_y = (G_y + G_y.T) / 2  # Ensure symmetric distance matrix

# # Marginals (mu and nu)
# mu = np.random.rand(n)
# mu /= mu.sum()  # Normalize to sum to 1
# nu = np.random.rand(m)
# nu /= nu.sum()  # Normalize to sum to 1

# # Solve the Gromov-Wasserstein problem
# T_optimal = solve_gw_problem(G_x, G_y, mu, nu)
# print("Optimal Transport Matrix (T):\n", T_optimal)

In [ ]:
# import numpy as np
# import nlopt


# # Define the Gromov-Wasserstein objective function
# def gromov_wasserstein_objective_nlopt(T_flat, G_x, G_y, n, m):
#     T = T_flat.reshape((n, m))
#     obj_value = 0
#     # Compute the GW cost
#     for i in range(n):
#         for j in range(m):
#             for k in range(n):
#                 for el in range(m):
#                     obj_value += T[i, k] * T[j, el] * (G_x[i, j] - G_y[k, el]) ** 2
#     return obj_value


# # Define the constraint function for row and column marginals
# def constraint_function(T_flat, mu, nu, n, m):
#     T = T_flat.reshape((n, m))
#     row_constraints = np.zeros(n)
#     col_constraints = np.zeros(m)

#     # Row constraints: sum_j T[i, j] = mu[i]
#     for i in range(n):
#         row_constraints[i] = np.sum(T[i, :]) - mu[i]

#     # Column constraints: sum_i T[i, j] = nu[j]
#     for j in range(m):
#         col_constraints[j] = np.sum(T[:, j]) - nu[j]

#     # Combine row and column constraints
#     return np.concatenate([row_constraints, col_constraints])


# # Set up and solve the Gromov-Wasserstein problem
# def solve_gw_problem(G_x, G_y, mu, nu):
#     n, m = G_x.shape  # Sizes of the two sets
#     x0 = np.outer(mu, nu).flatten()  # Initial guess: outer product of mu and nu
#     lower_bounds = np.zeros_like(x0)  # Ensure T[i, j] >= 0

#     # Set up the optimizer
#     opt = nlopt.opt(nlopt.LD_MMA, n * m)  # MMA is a good choice for non-convex problems
#     opt.set_min_objective(
#         lambda T_flat, grad: gromov_wasserstein_objective_nlopt(T_flat, G_x, G_y, n, m)
#     )
#     opt.set_lower_bounds(lower_bounds)

#     # Add row and column marginal constraints
#     opt.add_inequality_constraint(
#         lambda T_flat, grad: constraint_function(T_flat, mu, nu, n, m), 1e-8
#     )

#     # Solve the problem
#     try:
#         optimal_T_flat = opt.optimize(x0)
#         optimal_T = optimal_T_flat.reshape((n, m))  # Reshape to the transport matrix
#         return optimal_T
#     except RuntimeError as e:
#         print(f"NLopt error: {e}")
#         return None


# # Example usage
# n, m = 4, 4  # Size of the two sets (you can change this)
# # Random distance matrices (G_x and G_y)
# points_x = np.random.rand(n, 2)
# scale = 0
# points_y = points_x + scale * np.random.rand(m, 2)
# G_x = np.linalg.norm(points_x[:, None] - points_x, axis=-1)
# G_y = np.linalg.norm(points_y[:, None] - points_y, axis=-1)

# # Normalize the distance matrices (optional)
# G_x = (G_x + G_x.T) / 2  # Ensure symmetric distance matrix
# G_y = (G_y + G_y.T) / 2  # Ensure symmetric distance matrix

# # Marginals (mu and nu)
# mu = np.ones(n) / n
# mu /= mu.sum()  # Normalize to sum to 1
# nu = np.ones(m) / m
# nu /= nu.sum()  # Normalize to sum to 1

# # Solve the Gromov-Wasserstein problem
# T_optimal = solve_gw_problem(G_x, G_y, mu, nu)
# print("Optimal Transport Matrix (T):\n", T_optimal)

In [ ]:
# import nlopt
# import numpy as np


# # Define the Gromov-Wasserstein objective function with gradient
# def gromov_wasserstein_objective(T_flat, grad, G_x, G_y, n, m):
#     T = T_flat.reshape((n, m))
#     obj_value = 0
#     if grad.size > 0:
#         grad[:] = 0  # Initialize gradient to zero
#     # Compute the GW cost
#     for i in range(n):
#         for j in range(m):
#             for k in range(n):
#                 for el in range(m):
#                     cost = (G_x[i, j] - G_y[k, l]) ** 2
#                     obj_value += T[i, k] * T[j, l] * cost
#                     if grad.size > 0:
#                         grad[i * m + k] += T[j, el] * cost
#                         grad[j * m + el] += T[i, k] * cost
#                         grad[i * m + el] -= T[j, el] * cost
#                         grad[j * m + k] -= T[i, k] * cost
#     return obj_value


# # Example usage
# n, m = 4, 4  # Size of the two sets (you can change this)
# # Random distance matrices (G_x and G_y)
# points_x = np.random.rand(n, 2)
# scale = 0
# points_y = points_x + scale * np.random.rand(m, 2)
# G_x = np.linalg.norm(points_x[:, None] - points_x, axis=-1)
# G_y = np.linalg.norm(points_y[:, None] - points_y, axis=-1)

# # Normalize the distance matrices (optional)
# G_x = (G_x + G_x.T) / 2  # Ensure symmetric distance matrix
# G_y = (G_y + G_y.T) / 2  # Ensure symmetric distance matrix

# # Marginals (mu and nu)
# mu = np.ones(n) / n
# mu /= mu.sum()  # Normalize to sum to 1
# nu = np.ones(m) / m
# nu /= nu.sum()  # Normalize to sum to 1

# x0 = np.ones(n * m) / (n * m)  # Initial guess

# opt = nlopt.opt(nlopt.LD_MMA, n * m)  # MMA is a good choice for non-convex problems
# opt.set_min_objective(
#     lambda T_flat, grad: gromov_wasserstein_objective(T_flat, grad, G_x, G_y, n, m)
# )

# # Set lower bounds and other constraints if needed
# opt.set_lower_bounds(0)

# # Solve the problem
# try:
#     optimal_T_flat = opt.optimize(x0)
#     optimal_T = optimal_T_flat.reshape((n, m))  # Reshape to the transport matrix
#     print("Optimal Transport Matrix (T):\n", optimal_T)
# except RuntimeError as e:
#     print(f"NLopt error: {e}")